# HAH913E — ENMO over epochs (10s / 30s / 60s)

- **Public GitHub URL**: [https://github.com/mahouakon/HAH913E-ENMO](https://github.com/mahouakon/HAH913E-ENMO)
- **Team members**: PEHE, KONE
- **AI prompts used + modifications made afterwards**:
  - We asked **ChatGPT** to provide an **example of how to calculate ENMO** and integrate it over different epoch lengths.
  - We then **adapted this example to our own assignment**: we changed the CSV filename, recreated the notebook cells one by one in VS Code, and adjusted the code so it works with our own dataset.


In [14]:
from pathlib import Path
import pandas as pd


DATA_PATH = Path("./0_z.csv")  
df = pd.read_csv(DATA_PATH)
print(df.head())


                     # accelerometer data in g
t    x       y                               z
0.00 -0.0938 -0.0156                    0.9531
0.02 -0.0938 -0.0156                    0.9531
0.04 -0.0938 -0.0156                    0.9531
0.06 -0.0938 -0.0156                    0.9531


In [15]:
from pathlib import Path
import pandas as pd
import numpy as np

DATA_PATH = Path("./0_z.csv")

def load_data(path: Path) -> pd.DataFrame:
    """Charge le fichier CSV sans en-tête, ajoute les noms et convertit en numérique."""
    if not path.exists():
        raise FileNotFoundError(f"❌ Fichier introuvable : {path.resolve()}")

    # 🔸 Lecture du fichier en sautant la première ligne descriptive
    df = pd.read_csv(
        path,
        skiprows=1,                # ignore "# accelerometer data in g"
        names=["t", "x", "y", "z"] # noms des colonnes
    )

    # 🧹 Conversion des colonnes en numérique (remplace les valeurs invalides par NaN)
    for col in ["t", "x", "y", "z"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # 🧽 Suppression des lignes qui auraient des NaN (valeurs non convertibles)
    df = df.dropna(subset=["t", "x", "y", "z"])

    # 🔸 Tri et nettoyage
    df = df.sort_values("t").drop_duplicates(subset="t").reset_index(drop=True)
    return df

def compute_enmo(df: pd.DataFrame) -> pd.DataFrame:
    """Calcule la colonne ENMO à partir de x, y, z."""
    norm = np.sqrt(df["x"]**2 + df["y"]**2 + df["z"]**2)
    enmo = np.maximum(norm - 1.0, 0.0)
    out = df.copy()
    out["enmo"] = enmo
    return out

# 🚀 Exécution
df = load_data(DATA_PATH)
print("✅ Fichier chargé avec succès !")
print(f"Lignes : {len(df)}, Temps min = {df['t'].min()} s, Temps max = {df['t'].max()} s")
print(df.head())

df_enmo = compute_enmo(df)
print("\n✅ ENMO calculé — aperçu :")
print(df_enmo.head())


✅ Fichier chargé avec succès !
Lignes : 21000, Temps min = 0.0 s, Temps max = 419.98 s
      t       x       y       z
0  0.00 -0.0938 -0.0156  0.9531
1  0.02 -0.0938 -0.0156  0.9531
2  0.04 -0.0938 -0.0156  0.9531
3  0.06 -0.0938 -0.0156  0.9531
4  0.08 -0.0938 -0.0156  0.9531

✅ ENMO calculé — aperçu :
      t       x       y       z  enmo
0  0.00 -0.0938 -0.0156  0.9531   0.0
1  0.02 -0.0938 -0.0156  0.9531   0.0
2  0.04 -0.0938 -0.0156  0.9531   0.0
3  0.06 -0.0938 -0.0156  0.9531   0.0
4  0.08 -0.0938 -0.0156  0.9531   0.0


In [16]:
def integrate_enmo(df: pd.DataFrame, epoch_length: float) -> pd.DataFrame:
    """
    Intègre l'ENMO sur des époques de taille epoch_length (en secondes).
    Retourne un nouveau DataFrame avec les moyennes d'ENMO par époque.
    """
    df_epoch = df.copy()
    df_epoch["epoch"] = (df_epoch["t"] // epoch_length) * epoch_length
    df_grouped = df_epoch.groupby("epoch")["enmo"].mean().reset_index()
    df_grouped.rename(columns={"enmo": f"enmo_{int(epoch_length)}s"}, inplace=True)
    return df_grouped

# 🧮 Calcul pour 10, 30 et 60 secondes
df_10s = integrate_enmo(df_enmo, 10)
df_30s = integrate_enmo(df_enmo, 30)
df_60s = integrate_enmo(df_enmo, 60)

print(df_10s.head())
print(df_30s.head())
print(df_60s.head())


   epoch  enmo_10s
0    0.0   0.00004
1   10.0   0.00000
2   20.0   0.00000
3   30.0   0.00000
4   40.0   0.00000
   epoch  enmo_30s
0    0.0  0.000013
1   30.0  0.000117
2   60.0  0.000000
3   90.0  0.000000
4  120.0  0.000000
   epoch  enmo_60s
0    0.0  0.000065
1   60.0  0.000000
2  120.0  0.000133
3  180.0  0.000093
4  240.0  0.358817
